In [1]:
import initialize_notebook # noqa

from hslu.dlm03.common import agent, backend, chat, chat_display, tools, types
from hslu.dlm03.tools import lint
from hslu.dlm03.util import ipython_utils, ratelimit, unified_diff

In [11]:
from mcp.server import FastMCP
from pydantic import dataclasses
import docker
import secrets
import time

SERVER = FastMCP()
DOCKER_CLIENT = docker.from_env()

@SERVER.tool()
def password_generator(length: int):
    return secrets.token_urlsafe(length)

@SERVER.tool()
def docker_list_images():
    return DOCKER_CLIENT.images.list()

@SERVER.tool()
def docker_list_containers():
    return DOCKER_CLIENT.containers.list()

@SERVER.tool()
def docker_run_container(image: str, command: list[str] | None = None, ports: dict[str, str] = None, environment: dict[str, str] = None):
    """Runs a docker with the given image, and the given command to execute inside the container."""
    DOCKER_CLIENT.images.pull(image)
    container = DOCKER_CLIENT.containers.run(image, command, detach=True, environment=environment, ports=ports)
    return container.id

@SERVER.tool()
def docker_logs(container_id: str):
    container = DOCKER_CLIENT.containers.get(container_id)
    return container.logs()

@SERVER.tool()
def wait(seconds: float):
    time.sleep(seconds)
    return "Done"

In [3]:
import threading

import uvicorn

PORT = 5000
HOST = "localhost"

RUN_ARGS = {
    "app": SERVER.streamable_http_app,
    "port": PORT,
    "host": HOST,
}

MCP_THREAD = threading.Thread(target=uvicorn.run, kwargs=RUN_ARGS)
MCP_THREAD.start()

In [4]:
BACKEND = backend.Gemini2p5Flash().get_async_backend()

In [5]:
MCP_SERVER_URL = f"http://{HOST}:{PORT}/mcp"
TOOL_MANAGER = tools.ToolManager.from_url(MCP_SERVER_URL)

In [6]:
SYSTEM_INSTRUCTIONS = """You are an expert AI Agent, specializing in deploying software applications using Docker.
Your task is to use the tools available to you to fulfill the user's request. You should not ask the user any more information, try to figure out on your own.
"""

In [7]:
AGENT = agent.Agent(BACKEND, TOOL_MANAGER)

In [8]:
CHAT = chat.Chat(messages=[{"role": "system", "content": SYSTEM_INSTRUCTIONS}])
ipython_utils.display_agent(AGENT, chat=CHAT)

HTML(value='<style>\n.chat-container {\n    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto…

In [10]:
# Can you deploy a MySQL Database?